In [2]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

In [1]:
import torch
from transformers import BertConfig, BertTokenizer, BertTokenizerFast, BertForSequenceClassification
import onnxruntime as ort
from onnxruntime_tools import optimizer
import argparse
import pandas as pd
import numpy as np
# from transformers.convert_graph_to_onnx import convert
from convert_graph_to_onnx import convert
import os
import time
import torch.nn.functional as F
import onnx
from quantize import quantize, QuantizationMode

/scratch/da2734/pyenv_dval_wb_twitter/py3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:541: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/scratch/da2734/pyenv_dval_wb_twitter/py3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:542: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/scratch/da2734/pyenv_dval_wb_twitter/py3.7/lib/python3.7/site-packages/tensorboard/compat/tensorflow_stub/dtypes.py:543: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)]

In [9]:
# conversion
# model_path = '/scratch/da2734/twitter/jobs/onnx/results_simpletransformers_jun3_10Klabels_0_all_labels/lost_job_1mo/'
# onnx_path = '/scratch/da2734/twitter/jobs/onnx/results_simpletransformers_jun3_10Klabels_0_all_labels/lost_job_1mo/onnx/'

import shutil

for label in ["is_unemployed", "job_search", "is_hired_1mo", "job_offer", "lost_job_1mo"]:
    
#     if label == 'lost_job_1mo': continue

    print(label)
#     model_path = '/scratch/da2734/twitter/jobs/onnx/results_simpletransformers_jun3_10Klabels_0_all_labels/{}/'.format(label)
    model_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/pytorch_models/{}/'.format(label)

    onnx_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/onnx/{}/'.format(label)

#     os.rmdir(onnx_path)
    if os.path.isdir(onnx_path):
        shutil.rmtree(onnx_path)
    
    os.makedirs(onnx_path)
    
    print('converting')
    convert(framework="pt", 
        model=model_path, 
        tokenizer="DeepPavlov/bert-base-cased-conversational",
        output=onnx_path+'converted.onnx', 
        opset=11)

    print('optimizing')
    # ONNX optimization
    optimized_model = optimizer.optimize_model(onnx_path+'/converted.onnx',
                                               model_type='bert', 
                                               num_heads=12, 
                                               hidden_size=768)

    optimized_onnx_model_path = os.path.join(onnx_path, 'bert_optimized.onnx')
    optimized_model.save_model_to_file(optimized_onnx_model_path)
    print('Optimized model saved at :', optimized_onnx_model_path)
    
#     break

is_unemployed
converting
ONNX opset version set to: 11
Loading pipeline (model: /scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/pytorch_models/is_unemployed/, tokenizer: DeepPavlov/bert-base-cased-conversational)
PyTorch: 1.5.0+cu101
optimizing
Optimized model saved at : /scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/onnx/is_unemployed/bert_optimized.onnx
job_search
converting
ONNX opset version set to: 11
Loading pipeline (model: /scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/pytorch_models/job_search/, tokenizer: DeepPavlov/bert-base-cased-conversational)
PyTorch: 1.5.0+cu101
optimizing
Optimized model saved at : /scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/onnx/job_search/bert_optimized.onnx
is_hired_1mo
converting
ONNX opset version set to: 11
Loading pipeline (model: /scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/pytorch_models/is_hired_1mo/, tokenizer: DeepPavlov/bert-base-ca

# test inference

In [4]:
# inference
def get_tokens(tokens_dict, i):
    i_tokens_dict = dict()
    for key in ['input_ids','token_type_ids','attention_mask']:
        i_tokens_dict[key] = tokens_dict[key][i]
    tokens = {name: np.atleast_2d(value) for name, value in i_tokens_dict.items()}
    return tokens

def inference(onnx_model, model_dir, examples, fast_tokenizer, num_threads):
    quantized_str = ''
    if 'quantized' in onnx_model:
        quantized_str = 'quantized'
    onnx_inference = []
#     pytorch_inference = []
    # onnx session
    options = ort.SessionOptions()
    options.graph_optimization_level = ort.GraphOptimizationLevel.ORT_ENABLE_ALL
    options.execution_mode = ort.ExecutionMode.ORT_SEQUENTIAL
    options.intra_op_num_threads = 1
    print(onnx_model)
    ort_session = ort.InferenceSession(onnx_model, options)

    # pytorch pretrained model and tokenizer
    tokenizer = BertTokenizerFast.from_pretrained(model_dir)
    tokenizer_str = "BertTokenizerFast"

    print("**************** {} ONNX inference with batch tokenization and with {} tokenizer****************".format(quantized_str, tokenizer_str))
    start_onnx_inference_batch = time.time()
    start_batch_tokenization = time.time()
    tokens_dict = tokenizer.batch_encode_plus(examples, max_length=128)
    total_batch_tokenization_time = time.time() - start_batch_tokenization
    total_inference_time = 0
    total_build_label_time = 0
    for i in range(len(examples)):
        """
        Onnx inference with batch tokenization
        """
        tokens = get_tokens(tokens_dict, i)
        #inference
        start_inference = time.time()
        ort_outs = ort_session.run(None, tokens)
        total_inference_time = total_inference_time + (time.time() - start_inference)
        #build label
        start_build_label = time.time()
        torch_onnx_output = torch.tensor(ort_outs[0], dtype=torch.float32)
        onnx_logits = F.softmax(torch_onnx_output, dim=1)
        logits_label = torch.argmax(onnx_logits, dim=1)
        label = logits_label.detach().cpu().numpy()
#         onnx_inference.append(label[0])
        onnx_inference.append( onnx_logits.detach().cpu().numpy()[0].tolist() )
        total_build_label_time = total_build_label_time + (time.time() - start_build_label)
#         print(i, label[0], onnx_logits.detach().cpu().numpy()[0].tolist(), type(onnx_logits.detach().cpu().numpy()[0]) )

    end_onnx_inference_batch = time.time()
    print("Total batch tokenization time (in seconds): ", total_batch_tokenization_time)
    print("Total inference time (in seconds): ", total_inference_time)
    print("Total build label time (in seconds): ", total_build_label_time)
    print("Duration ONNX inference (in seconds) with {} and batch tokenization: ".format(tokenizer_str), end_onnx_inference_batch - start_onnx_inference_batch, (end_onnx_inference_batch - start_onnx_inference_batch)/len(examples))

#     print(onnx_inference)
    return onnx_inference


# model_path = '/scratch/da2734/twitter/jobs/onnx/results_simpletransformers_jun3_10Klabels_0_all_labels/{}/'.format(label)
# onnx_labels = inference(onnx_path+'converted.onnx', 
#                                         model_path, 
#                                         examples, 
#                                         fast_tokenizer=True, 
#                                         num_threads=1)

In [5]:
eval_df = pd.read_csv('/scratch/da2734/twitter/data/jun3_10Klabels/data_binary_pos_neg_unbalanced/train_lost_job_1mo.csv')
# eval_df.head()
eval_df = eval_df[['text', 'class']]
# print(eval_df.head())
print('Number of examples: ', eval_df.shape[0])
examples = eval_df.text.values.tolist()
# len(examples)
# examples[:1]
examples = examples[:10]
print(examples)

Number of examples:  6683
["I'm am currently lolling for a new job in either the video, media, or graphic design fields. If anyone has any leads or openings please let…", 'Can a NFL player who gets laid off file for Unemployment??? Lol', "My new job just called me cause they haven't heard from me in a few days uhhhh maybe it's cause you gave me one shift this week?", 'Unemployed? Buy An iPhone: By Reuven Gorsht, Vice-President, Customer Strategy, SAP http://t.co/bwOowJd9', 'I thought it was gunna get worse before it got better.. I was right when I got fired today', 'Also my co-workers "fired" me cause I didn\'t come into work yesterday', "It's a good time to be an #ExecutiveChef. It's an even better time to be an Executive Chef who's looking for a new job. Here's one opportunity (of many) available through @EHSGetHired  : https://t.co/VbQ60Bucup", 'liz &amp; i got hired at 2 diff places today &amp; we\'re like "ok let me use ur discount &amp; u can use mine ok ?"', "I've been unemploye

In [7]:
label = "is_unemployed"
onnx_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/onnx/{}/'.format(label)
model_path = '/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/pytorch_models/{}/'.format(label)
onnx_labels = inference(onnx_path+'converted.onnx', 
                                        model_path, 
                                       examples, 
                                        fast_tokenizer=True, 
                                        num_threads=1)

/scratch/da2734/twitter/jobs/running_on_200Msamples/iteration1/bert/onnx/is_unemployed/converted.onnx
****************  ONNX inference with batch tokenization and with BertTokenizerFast tokenizer****************
Total batch tokenization time (in seconds):  0.0011553764343261719
Total inference time (in seconds):  0.3947763442993164
Total build label time (in seconds):  0.0019130706787109375
Duration ONNX inference (in seconds) with BertTokenizerFast and batch tokenization:  0.39865732192993164 0.039865732192993164


In [8]:
onnx_labels

[[0.9606851935386658, 0.03931479528546333],
 [0.9667195081710815, 0.033280499279499054],
 [0.9853585958480835, 0.014641438610851765],
 [0.9842662811279297, 0.01573370397090912],
 [0.5481763482093811, 0.4518236219882965],
 [0.6770866513252258, 0.32291334867477417],
 [0.9867992997169495, 0.013200734741985798],
 [0.9336254596710205, 0.0663745179772377],
 [0.04140980914235115, 0.9585901498794556],
 [0.9710121750831604, 0.028987891972064972]]

# misc stuff

In [16]:
# import pyarrow.parquet as pq
# from glob import glob
# import os
# import numpy as np

SLURM_JOB_ID = 123123123
SLURM_ARRAY_TASK_ID = 10
SLURM_ARRAY_TASK_COUNT = 500

path_to_data='/scratch/spf248/twitter/data/classification/US/'
paths_to_filtered=list(np.array_split(
                        # glob(os.path.join(path_to_data,'filtered','*.parquet')),
                        glob(os.path.join(path_to_data,'filtered_10perct_sample','*.parquet')),
#                         glob(os.path.join(path_to_data,'filtered_1perct_sample','*.parquet')),
                        SLURM_ARRAY_TASK_COUNT)[SLURM_ARRAY_TASK_ID])
print('#files:', len(paths_to_filtered))

tweets_filtered=pd.DataFrame()
for file in paths_to_filtered:
    print(file)
    tweets_filtered=pd.concat([tweets_filtered,pd.read_parquet(file)[['tweet_id','text']]])
    print(tweets_filtered.shape)
    

tweets_filtered = tweets_filtered[:10]
print(tweets_filtered.head())
print(tweets_filtered.shape)

examples = tweets_filtered.text.values.tolist()
# len(examples)
# examples[:1]
# examples = examples[:10]
examples

#files: 1
/scratch/spf248/twitter/data/classification/US/filtered_10perct_sample/part-00011-e5ecb1c7-f08f-4246-a46d-2643ba831074-c000.snappy.parquet
(42990, 2)
             tweet_id                                               text
0  703470004939796480  @MarquesEvans @jcalv6 I'm proud of both of y'a...
1  703471678433538048  RT @agbxlwt: @MelanieLBBH my sister (@vesselhs...
2  703473306058358784  RT @BrysonTiIIller: relax, u'll graduate, u'll...
3  703473997854461952  Let's have a wrap party!!!! Message me to set ...
4  703475533300912129  #Job #Lexington Uber Full Time Driver - Indepe...
(10, 2)


["@MarquesEvans @jcalv6 I'm proud of both of y'all keep up the good work.",
 'RT @agbxlwt: @MelanieLBBH my sister (@vesselhs) drew you! please notice this 💓💓 her instagram is @/quiteartzy xx https://t.co/zzmmy9uyyS',
 "RT @BrysonTiIIller: relax, u'll graduate, u'll get a job, u'll become an adult, u'll find someone who loves you. u have an entire life. thi…",
 "Let's have a wrap party!!!! Message me to set up a party and get wrapped for free. #itworks… https://t.co/6VMYeUDMHm",
 '#Job #Lexington Uber Full Time Driver - Independent Contractor (Lexington) - Uber Driving Partners (Lexington,... https://t.co/LLkheC7o3e',
 'Montco Employer Of 1,000 Workers Expects Major Layoffs: Reports https://t.co/6kZCQR3EW5',
 '@firstagentarp ...That attack should /not/ be TPKing. Show me your party composition?',
 'RT @CarAutoDaily: Future Transport Networks... -  https://t.co/vL9JoFcZID',
 '#NeverTrump Our country worked too damn hard recovering from the last Republican disaster of a President!  https:

In [34]:
onnx_labels = inference(onnx_path+'converted.onnx', 
                                        model_path, 
                                       examples, 
                                        fast_tokenizer=True, 
                                        num_threads=1)
onnx_labels

/scratch/da2734/twitter/jobs/onnx/results_simpletransformers_jun3_10Klabels_0_all_labels/lost_job_1mo/onnx/converted.onnx
****************  ONNX inference with batch tokenization and with BertTokenizerFast tokenizer****************
Total batch tokenization time (in seconds):  0.000499725341796875
Total inference time (in seconds):  0.3943750858306885
Total build label time (in seconds):  0.0016336441040039062
Duration ONNX inference (in seconds) with BertTokenizerFast and batch tokenization:  0.39713454246520996 0.03971345424652099


[[0.9864523410797119, 0.013547663576900959],
 [0.9874812960624695, 0.012518745847046375],
 [0.9874129295349121, 0.01258703414350748],
 [0.9917016625404358, 0.00829832162708044],
 [0.9961840510368347, 0.003815914737060666],
 [0.9908242225646973, 0.009175794199109077],
 [0.9920403361320496, 0.007959687151014805],
 [0.9957973957061768, 0.00420261500403285],
 [0.9902395009994507, 0.009760518558323383],
 [0.9889292120933533, 0.011070745065808296]]

In [37]:
predictions_filtered_df = pd.DataFrame(data=onnx_labels, columns = ['first', 'second'])
predictions_filtered_df = predictions_filtered_df.set_index(tweets_filtered.tweet_id)

predictions_filtered_df

,first,second
tweet_id,,
703470004939796480,0.986452,0.013548
703471678433538048,0.987481,0.012519
703473306058358784,0.987413,0.012587
703473997854461952,0.991702,0.008298
703475533300912129,0.996184,0.003816
703476921569513472,0.990824,0.009176
703477601117929473,0.992040,0.007960
703477722534666240,0.995797,0.004203
703483250677669888,0.990240,0.009761
